# OOD Detection Pipeline

## 1. Load data to obtain hidden representations.

### Datasets

In [ ]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset


print("Loading Jordan dataset - train")
id_train_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_base_dataset"
)
print(f"Obtained {len(id_train_base_dataset)} training examples from Jordan dataset - train")

print("Loading Jordan dataset - test")
id_test_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_base_dataset",
    num_samples=80
)
print(f"Obtained {len(id_test_base_dataset)} testing examples from Jordan dataset - test")

print("Loading Maestro dataset - test")
ood_base_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_base_dataset",
    num_samples=80
)
print(f"Obtained {len(ood_base_dataset)} testing examples from Maestro dataset - test")

print("Example id_train_base_dataset[100]", id_train_base_dataset[100]["input_ids"][:10])
print("Example id_test_base_dataset[100]", id_test_base_dataset[50]["input_ids"][:10])
print("Example ood_base_dataset[100]", ood_base_dataset[50]["input_ids"][:10])


From each data sample in each of these datasets, extract all sliding windows of length SLIDING_WINDOW_LEN = 120 tokens (=40 events). This can be configured in constants/real_time_constants.py. The stride being STRIDE = 3 corresponds to moving the window by 3 tokens (=1 event) at a time. 

In [ ]:
from constants.real_time_constants import SLIDING_WINDOW_LEN, STRIDE
from data.sliding_window import SlidingWindowDataset


id_train_dataset = SlidingWindowDataset(id_train_base_dataset, name="id_train_dataset", k=SLIDING_WINDOW_LEN, stride=STRIDE)
id_test_dataset = SlidingWindowDataset(id_test_base_dataset, name="id_test_dataset", k=SLIDING_WINDOW_LEN, stride=STRIDE, num_samples=200)
ood_dataset = SlidingWindowDataset(ood_base_dataset, name="ood_dataset", k=SLIDING_WINDOW_LEN, stride=STRIDE, num_samples=200)

print("len(id_train_dataset)", len(id_train_dataset))
print("len(id_test_dataset)", len(id_test_dataset))
print("len(ood_dataset)", len(ood_dataset))

print("Example id_train_dataset[100]", id_train_dataset[100]["input_ids"][:10])
print("Example id_test_dataset[100]", id_test_dataset[100]["input_ids"][:50])
print("Example ood_dataset[100]", ood_dataset[100]["input_ids"][:50])


    

### DataLoaders

In [ ]:
from torch.utils.data import DataLoader
from utils.data_loading import collate_fn

batch_size = 8

id_train_dataloader = DataLoader(id_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
id_test_dataloader = DataLoader(id_test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
ood_dataloader = DataLoader(ood_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


### Model

In [ ]:
from constants.model_constants import DEVICE, JORDAN_MODEL_NAME
import torch
from transformers import AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained(
    JORDAN_MODEL_NAME,
    dtype=torch.float32,
).to(DEVICE)

# 2. OOD Detector

Refer to [OOD detection docs](../../docs/ood_detection.md) for more information about this.

In [ ]:
from main.transformation_functions import extract_layer_transformation, GaussianMixtureWithScore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from main.transformations import Transformations
from main.scoring_functions import identity
from extract_layers.pooling_functions import pool_mean_std

transformations = Transformations(
    [
        extract_layer_transformation(model, pool_mean_std, [17]),
        PCA(n_components=100),
        StandardScaler(),
        GaussianMixtureWithScore(
            n_components=4,
            dim=100,
            diagonal_cov=False,
        ), # fits a GMM with 4 components to the data, with full covariance matrix, then returns the negative log density.
    ]
)
scoring_function = identity # above transformations already return a score, no further scoring needed.

## Create OOD detector
from transformations and scoring function. Takes about 1 minute with STRIDE=30, lower STRIDE means longer.

In [ ]:
from main.ood_detector import OODDetector

ood_detector = OODDetector(
    embedding_function=transformations,
    scoring_function=scoring_function,
    id_train_data=id_train_dataloader,
)

We can evaluate the detector!

```threshold = 0.7``` with ```threshold_type = "percentile"``` means that the score threshold is set to be the 70th percentile of all the scores (ID test + OOD test combined).

In [ ]:
confusion_matrix, true_positive_rate, false_positive_rate = ood_detector.evaluate(
    id_test_dataloader,
    ood_dataloader,
    threshold=0.5,
    threshold_type="percentile",
)
print("Confusion matrix:", confusion_matrix)
print("True positive rate:", true_positive_rate)
print("False positive rate:", false_positive_rate)

f1_score = 2 * (true_positive_rate * false_positive_rate) / (true_positive_rate + false_positive_rate)

print("F1 score:", f1_score)

We can plot the ROC curve and get AUROC!

AUROC will appear as ```main/auroc.png```

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
from eval.auroc import get_auroc

auroc = get_auroc(
    ood_detector,
    id_test_dataloader,
    ood_dataloader,
)

In [ ]:
print("AUROC:", auroc)



## Visualization

Hover above the bars to see the sample IDs corresponding to the scores. In the next cell, you can input the IDs to hear what the samples sound like.

First make sure that the dataset shuffling is fixed.

In [ ]:
id_test_samples = torch.cat([batch["input_ids"] for batch in id_test_dataloader]).to(DEVICE)
ood_samples = torch.cat([batch["input_ids"] for batch in ood_dataloader]).to(DEVICE)

print("ID test samples:", id_test_samples.shape)
print("OOD samples:", ood_samples.shape)



In [ ]:
# sort by scores
id_test_samples = id_test_samples[torch.argsort(ood_detector.score(id_test_samples))]
ood_samples = ood_samples[torch.argsort(ood_detector.score(ood_samples))]


In [ ]:
from eval.graph_viz import get_graph_visualization

fig = get_graph_visualization(
    ood_detector,
    id_test_samples,
    ood_samples,
)

### Note: clear outputs for below cell if you run it before saving otherwise it will hang

In [ ]:
from utils.audio_grid import create_audio_grid

create_audio_grid(id_test_samples, max_samples=24, indices=[0, 5, 10, 15, 25, 35, 55, 75, 95, 115, 135, 155, 175, 180, 185, 190, 193, 194, 195, 196, 197, 198, 199, 200], cols=4)



Hear individual ID input samples and outputs:

In [ ]:
id_input_index = 190
ood_input_index = 1


In [ ]:
import os
from utils.convert import sequence_to_wav
from IPython.display import Audio

id_input_sample = id_test_samples[id_input_index].unsqueeze(0).to(DEVICE)
id_sequence = id_input_sample[0].tolist()
print("ID sequence (first 50 tokens):", id_sequence[:50])
print("OOD score for ID sequence", ood_detector.score(id_input_sample).item())
id_export_filepath = "id_input_sample.wav"
if os.path.exists(id_export_filepath):
    os.remove(id_export_filepath)
sequence_to_wav(id_sequence, id_export_filepath)



Audio(id_export_filepath)




















In [ ]:
ood_input_sample = ood_samples[ood_input_index].unsqueeze(0).to(DEVICE)
ood_sequence = ood_input_sample[0].tolist()
print("OOD sequence (first 50 tokens):", ood_sequence[:50])
print("OOD score for OOD sequence", ood_detector.score(ood_input_sample).item())
ood_export_filepath = "ood_input_sample.wav"
sequence_to_wav(ood_sequence, ood_export_filepath)

Audio(ood_export_filepath)

In [ ]:
from utils.sample import generate_tokens
from constants.token_constants import AR

generate_length = 120

id_output_sample = generate_tokens(
    model,
    [AR],
    id_sequence[1:],
    generate_length,
)

print("input", id_sequence[:50])
input_len = len(id_sequence) - 1
print("output", id_output_sample[input_len:input_len+50])

id_export_filepath = "id_output_sample.wav"
sequence_to_wav(id_output_sample, id_export_filepath)

Audio(id_export_filepath)

